# Step 1: Tokenizer and Vocabulary Creation (Add more words if needed)

## 1. Import Necessary Libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# Step 1: Tokenizer and Vocabulary Creation (Add more words if needed)
sentence_en = "I love AI ."
# sentence_fr = "J' adore l'IA ."
sentence_fr = "నాకు ధోని అంటే ఇష్టం ."

word_map_en = {"<pad>": 0, "I": 1, "love": 2, "AI": 3, ".": 4}
word_map_fr = {"<pad>": 0, "నాకు": 1, "ధోని": 2, "అంటే": 3, "ఇష్టం": 4, ".": 5}

## 2. Tokenizing sentences

In [3]:
# Tokenizing sentences
def tokenize(sentence, word_map):
    return torch.tensor([word_map[word] for word in sentence.split()])

## 3.Tokenize the input and target sentences

In [4]:
# Tokenize the input and target sentences
input_tensor = tokenize(sentence_en, word_map_en).unsqueeze(0)  # Shape (1, 6)
target_tensor = tokenize(sentence_fr, word_map_fr).unsqueeze(0)  # Shape (1, 6)

# Step 2: Positional Encoding

In [5]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(torch.log(torch.tensor(10000.0)) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        return x + self.encoding[:, :x.size(1)]

# Test Case 1 : Positional Encoding

In [6]:
# Positional Encoding Test Case
import torch
import torch.nn as nn
import numpy as np

def test_positional_encoding():
    d_model = 16  # Smaller model for testing
    max_len = 10  # Small sequence length
    pos_enc = PositionalEncoding(d_model, max_len)

    # Create a mock input tensor
    x = torch.zeros((1, max_len, d_model))  # Batch size = 1, seq_len = 10, d_model = 16

    # Forward pass
    output = pos_enc(x)

    # Check if the output shape is correct
    assert output.shape == x.shape, "Output shape mismatch"

    # Check if the encoding changes the input
    assert not torch.allclose(output, x), "Positional encoding is not modifying the input"

    # Check for consistency in encoding values
    encoding1 = pos_enc.encoding[:, :max_len].detach().numpy()
    encoding2 = pos_enc.encoding[:, :max_len].detach().numpy()
    assert np.allclose(encoding1, encoding2), "Positional encoding is not consistent across calls"

    print("Test Case 1 :All positional encoding tests passed!")

# Run test case
test_positional_encoding()


Test Case 1 :All positional encoding tests passed!


# Step 3: Multi-Head Attention

In [7]:
# Step 3: Multi-Head Attention
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.d_k = d_model // num_heads
        self.d_v = d_model // num_heads

        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)

        self.fc = nn.Linear(d_model, d_model)

    def forward(self, x, mask=None):
        batch_size, seq_len, _ = x.size()

        q = self.query(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        k = self.key(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        v = self.value(x).view(batch_size, seq_len, self.num_heads, self.d_v).transpose(1, 2)

        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.d_k, dtype=torch.float))
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, float('-inf'))  # Ensure mask matches input length
        attn_weights = F.softmax(attn_scores, dim=-1)

        attention_output = torch.matmul(attn_weights, v).transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
        output = self.fc(attention_output)

        return output

# Test Case 2 Check multi_head_attention

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

def test_multi_head_attention():
    d_model = 16  # Small model for testing
    num_heads = 4  # Number of heads
    seq_len = 5  # Sequence length
    batch_size = 2  # Batch size

    mha = MultiHeadAttention(d_model, num_heads)

    # Create a random input tensor
    x = torch.rand((batch_size, seq_len, d_model))

    # Create a random mask
    mask = torch.randint(0, 2, (batch_size, num_heads, seq_len, seq_len))

    # Forward pass
    output = mha(x, mask)

    # Test 1: Check multi-head splits
    q = mha.query(x).view(batch_size, seq_len, num_heads, d_model // num_heads).transpose(1, 2)
    assert q.shape == (batch_size, num_heads, seq_len, d_model // num_heads), "Query split is incorrect"

    # Test 2: Check softmax attention weights sum to 1
    attn_scores = torch.matmul(q, q.transpose(-2, -1)) / torch.sqrt(torch.tensor(d_model // num_heads, dtype=torch.float))
    attn_weights = F.softmax(attn_scores, dim=-1)
    attn_sums = attn_weights.sum(dim=-1)
    assert torch.allclose(attn_sums, torch.ones_like(attn_sums), atol=1e-5), "Attention weights do not sum to 1"

    print("Test Case 2:All Multi-Head Attention tests passed!")

# Run test case
test_multi_head_attention()


Test Case 2:All Multi-Head Attention tests passed!


# Step 4: Feed-Forward Network

In [9]:
# Step 4: Feed-Forward Network
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=512):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))

# Step 5: Encoder Layer

In [10]:
# Step 5: Encoder Layer
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff=512):
        super(EncoderLayer, self).__init__()
        self.multihead_attn = MultiHeadAttention(d_model, num_heads)
        self.feedforward = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x, mask=None):
        attn_output = self.multihead_attn(x, mask)
        x = self.norm1(x + attn_output)
        ff_output = self.feedforward(x)
        x = self.norm2(x + ff_output)
        return x

# Test Case 3: Check Encoder Layer

In [11]:
import torch
import torch.nn as nn

def test_encoder_layer():
    d_model = 16  # Model dimension
    num_heads = 4  # Number of attention heads
    seq_len = 5  # Sequence length
    batch_size = 2  # Batch size

    encoder_layer = EncoderLayer(d_model, num_heads)

    # Create a random input tensor
    x = torch.rand((batch_size, seq_len, d_model))

    # Create a random mask
    mask = torch.randint(0, 2, (batch_size, num_heads, seq_len, seq_len))

    # Forward pass
    output = encoder_layer(x, mask)

    # Test 1: Check that MultiHeadAttention modifies input
    attn_output = encoder_layer.multihead_attn(x, mask)
    assert not torch.equal(attn_output, x), "MultiHeadAttention does not modify input"

    # Test 2: Check that MultiHeadAttention modifies input
    x = torch.rand(10, 20, d_model)
    output = encoder_layer(x)
    assert torch.all(output.std(dim=-1) > 0), "LayerNorm not applied correctly!"

    print("Test Case 3:EncoderLayer tests passed!")

# Run test case
test_encoder_layer()


Test Case 3:EncoderLayer tests passed!


# Step 6: Decoder Layer

In [12]:
# Step 6: Decoder Layer
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff=512):
        super(DecoderLayer, self).__init__()
        self.multihead_attn1 = MultiHeadAttention(d_model, num_heads)
        self.multihead_attn2 = MultiHeadAttention(d_model, num_heads)
        self.feedforward = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)

    def forward(self, x, encoder_output, tgt_mask=None, src_mask=None):
        attn_output1 = self.multihead_attn1(x, tgt_mask)
        x = self.norm1(x + attn_output1)
        attn_output2 = self.multihead_attn2(x, src_mask)
        x = self.norm2(x + attn_output2)
        ff_output = self.feedforward(x)
        x = self.norm3(x + ff_output)
        return x

# Test Case 4 : Decoder Layer

In [13]:

def test_decoder_layer():
    d_model = 512
    num_heads = 8
    seq_length = 10
    batch_size = 4

    # Create a random tensor as input
    x = torch.rand(batch_size, seq_length, d_model)
    encoder_output = torch.rand(batch_size, seq_length, d_model)

    # Initialize the DecoderLayer
    decoder_layer = DecoderLayer(d_model, num_heads)
    assert hasattr(decoder_layer, "multihead_attn2"), "DecoderLayer must have a cross-attention layer"
    assert hasattr(decoder_layer, "feedforward"), "DecoderLayer must have a feed-forward network"
    # Forward pass
    output = decoder_layer(x, encoder_output)

    # Assertions
    assert output.shape == x.shape, "Output shape mismatch"
    assert not torch.isnan(output).any(), "Output contains NaN values"

# Run the test
test_decoder_layer()
print("Test Case 4 Passed")

Test Case 4 Passed


# Step 7: Transformer Model

In [14]:
# Step 7: Transformer Model
class Transformer(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, num_encoder_layers, num_decoder_layers, max_len=5000):
        super(Transformer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_len)
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads) for _ in range(num_encoder_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads) for _ in range(num_decoder_layers)])
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, src, tgt, tgt_mask=None):
        src = self.pos_encoder(self.embedding(src))
        tgt = self.pos_encoder(self.embedding(tgt))

        encoder_output = src
        for layer in self.encoder_layers:
            encoder_output = layer(encoder_output)

        decoder_output = tgt
        for layer in self.decoder_layers:
            decoder_output = layer(decoder_output, encoder_output, tgt_mask=tgt_mask)

        output = self.fc_out(decoder_output)
        return output

# Step 8: Prediction Module

In [15]:
# Step 8: Prediction Module
def translate(input_sentence, word_map_en, word_map_fr, transformer):
    # Tokenize input sentence
    input_tensor = tokenize(input_sentence, word_map_en).unsqueeze(0)  # Shape (1, seq_len)

    # Generate a mask for the target sentence
    tgt_mask = torch.tril(torch.ones((input_tensor.size(1), input_tensor.size(1)))).unsqueeze(0).unsqueeze(0)  # Lower triangular mask

    # Initialize a tensor for the target sentence
    target_tensor = torch.zeros((1, input_tensor.size(1)), dtype=torch.long)

    # Predict the output sentence (translation)
    output = transformer(input_tensor, target_tensor, tgt_mask)

    # Apply Softmax to get probabilities
    softmax_output = F.softmax(output, dim=-1)

    # Get predicted token indices (Argmax)
    predicted_tokens = torch.argmax(softmax_output, dim=-1)

    # Convert predicted tokens back to words
    reverse_word_map_fr = {v: k for k, v in word_map_fr.items()}
    translated_sentence = [reverse_word_map_fr[token.item()] for token in predicted_tokens[0] if token != 0]

    return " ".join(translated_sentence)

In [16]:
# Step 9: Initialize Model
vocab_size_en = len(word_map_en)
vocab_size_fr = len(word_map_fr)
d_model = 128
num_heads = 8
num_encoder_layers = 2
num_decoder_layers = 2

transformer = Transformer(vocab_size_en, d_model, num_heads, num_encoder_layers, num_decoder_layers)

# Step 10: Take Input Sentence and Generate Translation

In [17]:
# Step 10: Take Input Sentence and Generate Translation
input_sentence = "I love AI ."  # Example input sentence
predicted_sentence = translate(input_sentence, word_map_en, word_map_fr, transformer)
print("Input Sentence:", input_sentence)
print("Predicted Translation:", predicted_sentence)

Input Sentence: I love AI .
Predicted Translation: ధోని ధోని ధోని ధోని
